# **Turning Point Example**

In [1]:
import numpy as np
import pandas as pd

import tournament_simulations.data_structures as ds
import tournament_simulations.permutations as pmt
import tournament_simulations.schedules.round_robin as rr

import turning_point.normal_coefficient as nc
import turning_point.permutation_coefficient as pc
import turning_point.variance_stats as vs

## **Tournament Matches**

In [2]:
matches = ds.Matches(
    pd.DataFrame(
        {
            "id": ["0", "0", "1", "1", "1", "1"],
            "date number": [0, 0, 0, 0, 1, 1],
            "home": ["a", "b", "A", "A", "C", "B"],
            "away": ["b", "a", "C", "B", "B", "A"],
            "winner": ["h", "a", "h", "h", "h", "h"],
        }
    )
)
matches.df

home away winner
id date number                 
0  0              a    b      h
   0              b    a      a
1  0              A    C      h
   0              A    B      h
   1              C    B      h
   1              B    A      h

In [3]:
ppm = ds.PointsPerMatch.from_home_away_winner(matches.home_away_winner)
ppm.df

team  points
id date number             
0  0              a       3
   0              b       0
   0              b       0
   0              a       3
1  0              A       3
   0              C       0
   0              A       3
   0              B       0
   1              C       3
   1              B       0
   1              B       3
   1              A       0

In [4]:
ppm.rankings

points
id team        
0  a          6
   b          0
1  A          6
   B          3
   C          3

## **Variances**

In [5]:
num_iteration_simulation = (2, 3) # 6 simulations: 2 batches of 3

id_to_probabilities = pd.Series(
    index=pd.Index(["0", "1"], name="id"), 
    data=[(1, 0, 0), (0, 0, 1)]
)

variance_stats = vs.ExpandingVarStats.from_matches(
    matches, num_iteration_simulation, id_to_probabilities
)
variance_stats.df

real var  mean  0.950-quantile
id final date                                
0  0               18.0   0.0             0.0
1  0               12.0   3.0             3.0
   1                3.0   3.0             3.0

## **Turning Point**

In [6]:
turning_point = nc.TurningPoint.from_expanding_var_stats(variance_stats)
turning_point.df

,turning point,%turning point
id,,
0,0.0,1.0
1,inf,inf


## **Permutated Turning Point**

In [7]:
from synthetic_tournaments import PermutationScheduler
scheduler = PermutationScheduler.from_matches(matches)
permutation_creator = pmt.MatchesPermutations(matches, scheduler)

permuted_matches = permutation_creator.create_n_permutations(n=2)
permuted_matches.df

home away winner
id  date number                 
0@0 0              a    b      h
    0              b    a      a
0@1 0              a    b      h
    0              b    a      a
1@0 0              C    B      h
    0              A    C      h
    1              B    A      h
    1              A    B      h
1@1 0              C    B      h
    0              A    C      h
    1              A    B      h
    1              B    A      h

In [8]:
permuted_var_stats = vs.ExpandingVarStats.from_matches(permuted_matches, num_iteration_simulation)
permuted_tp = pc.PermutationTurningPoint.from_expanding_var_stats(permuted_var_stats)
permuted_tp.df

,turning point,%turning point
id,,
0@0,inf,inf
0@1,inf,inf
1@0,inf,inf
1@1,inf,inf


In [9]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    permuted_statistics = permuted_tp.statistical_measures([2.5, 50, 97.5])
permuted_statistics

turning point                    %turning point                   
            mean std 2.5% 50% 97.5%           mean std 2.5% 50% 97.5%
id                                                                   
0            inf NaN  NaN NaN   NaN            inf NaN  NaN NaN   NaN
1            inf NaN  NaN NaN   NaN            inf NaN  NaN NaN   NaN

## **Comparison**

In [10]:
comparison = pc.TurningPointComparison(turning_point, permuted_tp)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    tp_comparison = comparison.comparison([2.5, 50, 97.5])
tp_comparison

turning point                         %turning point                    \
          normal mean std 2.5% 50% 97.5%         normal mean std 2.5% 50%   
id                                                                          
0            0.0  inf NaN  NaN NaN   NaN            1.0  inf NaN  NaN NaN   
1            inf  inf NaN  NaN NaN   NaN            inf  inf NaN  NaN NaN   

          
   97.5%  
id        
0    NaN  
1    NaN